# Featurestore - Mars21
## Feature retrieval (online & batch)
Christos Aniftos \
Soeren Petersen

In [ ]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [ ]:
PROJECT_ID = "PROJECT_ID" # Change to your project id
LOCATION = "us-central1" 
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "universe"
ENTITY="planets"

## Define clinets for FS admin and data management

In [ ]:
# Create admin_client for CRUD and data_client for reading feature values.
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

data_client = FeaturestoreOnlineServingServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

In [ ]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

## Read Values from FS Online Storage - Real time!

In [ ]:
feature_selector = feature_selector_pb2.FeatureSelector(
    id_matcher=feature_selector_pb2.IdMatcher(
        ids=["avg_max_temp_5d", "arr_max_temp_3d", "max_temp_std"]))

In [ ]:
%%time
data_client.read_feature_values(
    featurestore_online_service_pb2.ReadFeatureValuesRequest(
        entity_type=ENTITY_PATH,
        entity_id="mars",
        feature_selector=feature_selector))

In [ ]:
%%time
response_stream = data_client.streaming_read_feature_values(
    featurestore_online_service_pb2.StreamingReadFeatureValuesRequest(
        entity_type=ENTITY_PATH,
        entity_ids=["mars", "jupyter"],
        feature_selector=feature_selector))

for response in response_stream:
  print(response)

In [ ]:
EXPORT_TABLE_URI = "bq://feature-store-mars21.training.planet_data_v5"
FEATURE_REQ_CSV_PATH = "gs://feature-store-mars21/feature-request.csv"

In [ ]:
batch_serving_request = featurestore_service_pb2.BatchReadFeatureValuesRequest(
    featurestore=FS_PATH,
    csv_read_instances=io_pb2.CsvSource(
        gcs_source=io_pb2.GcsSource(uris=[FEATURE_REQ_CSV_PATH])),
    
    # Output info
    destination=featurestore_service_pb2.FeatureValueDestination(
        bigquery_destination=io_pb2.BigQueryDestination(
            # output to BigQuery table
            output_uri=EXPORT_TABLE_URI)),
    #destination=featurestore_service_pb2.FeatureValueDestination(
    #    tfrecord_destination=io_pb2.CsvDestination(
    #        gcs_destination=EXPORT_TF_PATH)),
   

    # Select features to read
    entity_type_specs=[
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # read feature values of features subscriber_type and duration_minutes from "bikes"
            entity_type_id=ENTITY, 
            feature_selector=feature_selector
        ),
        #featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
        #    entity_type_id="spaceships",
        #    feature_selector=feature_selector_pb2.FeatureSelector(
        #        id_matcher=feature_selector_pb2.IdMatcher(
        #            ids=["size", "speed"])))
    ])

In [ ]:
%%time
try:
    print(admin_client.batch_read_feature_values(batch_serving_request).result())
except Exception as ex:
    print(ex)